In [724]:
import pandas as pd
import numpy as np

import country_converter as coco

def clean_country_names(df, column='country', new_column='country_cleaned', manual_fixes=None):
    # Step 1: Basic cleanup
    df[column] = df[column].astype(str).str.strip()
    
    # Step 2: Try to convert names safely
    converted_names = []
    for name in df[column]:
        try:
            converted = coco.convert(names=name, to='name_short', not_found=None)
            converted_names.append(converted)
        except Exception as e:
            print(f"Error converting {name}: {e}")
            converted_names.append(None)
    
    # Step 3: Assign new column
    df[new_column] = converted_names
    
    # Step 4: Handle unmatched with manual fixes
    if manual_fixes is not None:
        df[new_column] = df[new_column].fillna(df[column].map(manual_fixes))
    
    # Step 5: Optional feedback
    unmatched = df[df[new_column].isna()]
    if not unmatched.empty:
        print("❗ Unmatched country names:", unmatched[column].unique())
    
    return df


In [727]:
child = pd.read_csv(r"C:\Users\eabro\Documents\NSS\Python\Projects\Utopia\EmilyBroh_Capstone\emilybroh_capstone\Category_A\child_mortality_norm_withallpts_onlycountries.csv")

In [729]:
regions_to_drop = ['World']
child = child[~child['country'].isin(regions_to_drop)]

In [731]:
import country_converter as coco

child['country_c'] = coco.convert(names=child['country'], to='name_short', not_found=None)


In [733]:
child.columns

Index(['country', 'year', 'under_5yo_deaths', 'norm_deaths_pct',
       'norm_with_pts', 'norm_pts_log5', 'country_c'],
      dtype='object')

In [735]:
cook = pd.read_csv(r"C:\Users\eabro\Documents\NSS\Python\Projects\Utopia\EmilyBroh_Capstone\emilybroh_capstone\Category_A\cook_withallpts_onlycountries.csv")

In [737]:
import country_converter as coco

cook['country_c'] = coco.convert(names=cook['country'], to='name_short', not_found=None)

In [739]:
cook.columns

Index(['country', 'year', 'clean_fuel_cooking', 'norm_clean_cooking',
       'norm_with_points', 'norm_with_points_log5', 'country_c'],
      dtype='object')

In [741]:
child_cook = pd.merge(cook, child, on = 'country_c', how = 'inner')
child_cook.shape

(191, 13)

In [743]:
child_cook['child_cook_pts'] = child_cook['norm_with_points_log5'] + child_cook['norm_pts_log5']

In [745]:
cc = child_cook[['country_c', 'child_cook_pts']]

In [747]:
elec = pd.read_csv(r"C:\Users\eabro\Documents\NSS\Python\Projects\Utopia\EmilyBroh_Capstone\emilybroh_capstone\Category_A\electric_with_allpts_onlycountries.csv")

In [749]:
regions_to_drop = ['World']
child = child[~child['country'].isin(regions_to_drop)]

In [752]:
import country_converter as coco

# Manual fixes for unmatched or non-standard names
manual_fixes = {
    'Channel Islands': 'United Kingdom'
}

# Function to clean country names
def clean_country_names(df, column='country', new_column='country_cleaned', manual_fixes=None):
    df[column] = df[column].astype(str).str.strip()
    
    converted_names = []
    for name in df[column]:
        try:
            converted = coco.convert(names=name, to='name_short', not_found=None)
            converted_names.append(converted)
        except Exception:
            converted_names.append(None)
    
    df[new_column] = converted_names

    if manual_fixes:
        df[new_column] = df[new_column].fillna(df[column].map(manual_fixes))
    
    unmatched = df[df[new_column].isna()]
    if not unmatched.empty:
        print("❗ Unmatched country names:", unmatched[column].unique())

    return df

# ✅ Apply to the elec dataframe
elec = clean_country_names(elec, manual_fixes=manual_fixes)



Channel Islands not found in regex
World not found in regex


In [753]:
elec.columns

Index(['country', 'year', 'access_electricity', 'norm_access_electricity',
       'norm_with_pts', 'norm_pts_log5', 'country_cleaned'],
      dtype='object')

In [754]:
food = pd.read_csv(r"C:\Users\eabro\Documents\NSS\Python\Projects\Utopia\EmilyBroh_Capstone\emilybroh_capstone\Category_A\food_insecure_withallpts_onlycountries.csv")

In [755]:
import country_converter as coco

food['country_c'] = coco.convert(names=food['country'], to='name_short', not_found=None)


Melanesia not found in regex


In [756]:
elec_food = pd.merge(elec, food, on ='country_c', how = 'inner')

KeyError: 'country_c'

In [ ]:
elec_food['elec_food'] = elec_food['norm_pts_log5'] + elec_food['norm_withpts_log5']

In [ ]:
ef = elec_food[['country_c', 'elec_food']]

In [ ]:
ccef = pd.merge(cc, ef, on = 'country_c', how = 'inner')

In [ ]:
ccef['ccef_pts'] = ccef['child_cook_pts'] + ccef['elec_food']
ccef

In [ ]:
life = pd.read_csv(r"C:\Users\eabro\Documents\NSS\Python\Projects\Utopia\EmilyBroh_Capstone\emilybroh_capstone\Category_A\life_expect_withallpts_onlycountries.csv")

In [ ]:
regions_to_drop = [
    'Northern America',
    'Oceania',
    'Europe',
    'Americas',
    'Asia',
    'Africa'
]

life = life[~life['country'].isin(regions_to_drop)]


In [ ]:
import country_converter as coco

life['country_c'] = coco.convert(names=life['country'], to='name_short', not_found=None)


In [763]:
life['norm_life_expect'] = (life['norm_life_expect'] - life['norm_life_expect'].min()) / (life['norm_life_expect'].max() - life['norm_life_expect'].min())

In [765]:
life['life_expectpts_0412'] = life['norm_life_expect'] * 32

In [767]:
l = life[['country_c', 'life_expectpts_0412']]

In [769]:
mat = pd.read_csv(r"C:\Users\eabro\Documents\NSS\Python\Projects\Utopia\EmilyBroh_Capstone\emilybroh_capstone\Category_A\maternal_mortality_withallpts_onlycountries.csv")

In [771]:
import country_converter as coco

mat['country_c'] = coco.convert(names=mat['country'], to='name_short', not_found=None)


In [773]:
mat = mat.sort_values('maternal_mortality', ascending = True)

In [775]:
mat['norm_maternal_mortality'] = 1 - (mat['norm_maternal_mortality'] - mat['norm_maternal_mortality'].min()) / (mat['norm_maternal_mortality'].max() - mat['norm_maternal_mortality'].min())

In [777]:
mat = mat.drop('norm_with_pts', axis = 1)

In [779]:
mat['norm_withpts_log5'] = mat['norm_maternal_mortality'] * 32

In [781]:
mat_life = pd.merge(mat, l, on = 'country_c', how = 'inner')
mat_life['mat_life'] = mat_life['norm_withpts_log5'] + mat_life['life_expectpts_0412']

In [783]:
ml = mat_life[['country_c', 'mat_life']]

In [785]:
ccefml = pd.merge(ccef, ml, on = 'country_c', how = 'inner')
ccefml

,country_c,child_cook_pts,elec_food,ccef_pts,mat_life
0,Italy,63.693271,64.000000,127.693271,61.667133
1,New Zealand,63.239312,59.634200,122.873512,60.814914
2,Japan,63.846636,63.428685,127.275320,62.727891
3,Chile,62.636079,58.936831,121.572910,59.144122
4,Romania,62.292543,58.468659,120.761201,55.192699
...,...,...,...,...,...
142,Uganda,16.034681,14.079473,30.114154,39.685838
143,Sierra Leone,3.546433,6.387274,9.933707,26.858615
144,Liberia,15.241006,6.955785,22.196791,26.698411
145,Burundi,13.932234,6.845133,20.777368,30.869472


In [787]:
ccefml['ccefml_pts'] = ccefml['ccef_pts'] + ccefml['mat_life']
ccefml

,country_c,child_cook_pts,elec_food,ccef_pts,mat_life,ccefml_pts
0,Italy,63.693271,64.000000,127.693271,61.667133,189.360405
1,New Zealand,63.239312,59.634200,122.873512,60.814914,183.688426
2,Japan,63.846636,63.428685,127.275320,62.727891,190.003211
3,Chile,62.636079,58.936831,121.572910,59.144122,180.717032
4,Romania,62.292543,58.468659,120.761201,55.192699,175.953901
...,...,...,...,...,...,...
142,Uganda,16.034681,14.079473,30.114154,39.685838,69.799992
143,Sierra Leone,3.546433,6.387274,9.933707,26.858615,36.792322
144,Liberia,15.241006,6.955785,22.196791,26.698411,48.895202
145,Burundi,13.932234,6.845133,20.777368,30.869472,51.646840


In [789]:
pol = pd.read_csv(r"C:\Users\eabro\Documents\NSS\Python\Projects\Utopia\EmilyBroh_Capstone\emilybroh_capstone\Category_A\pollution_withallpts_onlycountries.csv")

In [791]:
pol['norm_pm_value'] = 1 - (pol['norm_pm_value'] - pol['norm_pm_value'].min()) / pol['norm_pm_value'].max() - pol['norm_pm_value'].min()

In [793]:
pol['norm_with_pts_log5'] = pol['norm_pm_value'] * 32

In [795]:
slums = pd.read_csv(r"C:\Users\eabro\Documents\NSS\Python\Projects\Utopia\EmilyBroh_Capstone\emilybroh_capstone\Category_A\slums_norm_withbothpts_onlycountries.csv")

In [797]:
import country_converter as coco

slums['country_c'] = coco.convert(names=slums['country'], to='name_short', not_found=None)

In [799]:
import country_converter as coco

pol['country_c'] = coco.convert(names= pol['country'], to='name_short', not_found =None)

In [801]:
pol_slum = pd.merge(pol, slums, on ='country_c', how = 'inner')

In [803]:
ccefmlps = pd.merge(ccefml, pol_slum, on = 'country_c', how = 'inner')

In [805]:
ccefmlps['ccefmlps_pts'] = ccefmlps['ccefml_pts'] + ccefmlps['norm_with_pts_log5'] + ccefmlps['norm_log5']

In [807]:
ccefmlps = ccefmlps[['country_c', 'ccefmlps_pts']]

In [809]:
ccefmlps = ccefmlps.sort_values('ccefmlps_pts', ascending = False)

In [811]:
ccefmlps.to_csv('ccefmlps.csv', index=False, header=True)

# Something weird is going on when I try to read in the Sanitize CSV. Need to check this out later.

san = pd.read_csv(r"C:\Users\eabro\Documents\NSS\Python\Projects\Utopia\EmilyBroh_Capstone\emilybroh_capstone\Category_A\sanitize_withallpts_onlycountries.csv")

In [814]:
under = pd.read_csv(r"C:\Users\eabro\Documents\NSS\Python\Projects\Utopia\EmilyBroh_Capstone\emilybroh_capstone\Category_A\undernourish_norm_with_pts_countryonly.csv")

In [816]:
regions_to_drop = [
    'Melanesia',
    'Polynesia'
]

under = under[~under['country'].isin(regions_to_drop)]


In [818]:
import country_converter as coco

under['country_c'] = coco.convert(names= under['country'], to='name_short', not_found =None)

In [820]:
ccefmlpsu = pd.merge(under,ccefmlps, on = 'country_c', how = 'inner')

In [822]:
import country_converter as coco

ccefmlps['country_c'] = coco.convert(names=ccefmlps['country_c'], to='name_short', not_found=None)


In [824]:
ccefmlps

,country_c,ccefmlps_pts
35,Kuwait,244.307711
16,United Arab Emirates,236.740303
30,South Korea,230.185262
0,Italy,227.573026
18,Israel,225.434466
...,...,...
128,DR Congo,71.424128
138,Sierra Leone,67.860229
130,Somalia,63.062986
123,Chad,40.441653


In [826]:
water = pd.read_csv(r"C:\Users\eabro\Documents\NSS\Python\Projects\Utopia\EmilyBroh_Capstone\emilybroh_capstone\Category_A\water_withallpts_onlycountries.csv")
water.columns

Index(['country', 'year', 'water_pct', 'norm_water', 'norm_water_withpts',
       'norm_water_withpts_log5'],
      dtype='object')

In [828]:
pip install thefuzz[speedup]

Note: you may need to restart the kernel to use updated packages.


In [829]:
pip install country_converter

Note: you may need to restart the kernel to use updated packages.


In [832]:
import country_converter as coco

water['country'] = coco.convert(names=water['country'], to='name_short', not_found =None)


 Curaco not found in regex
 Channel Islands not found in regex
 Samoa not found in regex
 Congo not found in regex


In [834]:
import country_converter as coco

# Manual fixes for countries not properly handled by coco
manual_fixes = {
    'Channel Islands': 'United Kingdom',
    'Samoa': 'Samoa',
    'Congo': 'Congo, Rep.',
}

# List of regional labels to exclude
regions_to_drop = [
    'Northern America',
    'Oceania',
    'Europe',
    'Americas',
    'Asia',
    'Africa'
]

# Clean function for standardizing country names
def clean_country_names(df, column='country', new_column='country_c', manual_fixes=None, drop_regions=None):
    df[column] = df[column].astype(str).str.strip()
    
    converted_names = []
    for name in df[column]:
        try:
            converted = coco.convert(names=name, to='name_short', not_found=None)
            converted_names.append(converted)
        except Exception:
            converted_names.append(None)
    
    df[new_column] = converted_names

    if manual_fixes:
        df[new_column] = df[new_column].fillna(df[column].map(manual_fixes))
    
    # Drop region-level labels if provided
    if drop_regions:
        df = df[~df[column].isin(drop_regions)]

    # Print any remaining unmatched names
    unmatched = df[df[new_column].isna()]
    if not unmatched.empty:
        print("❗ Unmatched country names:", unmatched[column].unique())

    return df

# ✅ Run on your `water` dataframe
water = clean_country_names(
    water,
    column='country',
    new_column='country_c',
    manual_fixes=manual_fixes,
    drop_regions=regions_to_drop
)


Curaco not found in regex
Channel Islands not found in regex


In [835]:
#ccefmlps

In [836]:
water

,country,year,water_pct,norm_water,norm_water_withpts,norm_water_withpts_log5,country_c
0,Gibraltar,2018.0,100.000000,1.000000,5.000000,32.000000,Gibraltar
1,Monaco,2018.0,100.000000,1.000000,5.000000,32.000000,Monaco
2,Singapore,2018.0,100.000000,1.000000,5.000000,32.000000,Singapore
3,Liechtenstein,2018.0,100.000000,1.000000,5.000000,32.000000,Liechtenstein
4,San Marino,2018.0,100.000000,1.000000,5.000000,32.000000,San Marino
...,...,...,...,...,...,...,...
220,Sierra Leone,2018.0,31.103751,0.030482,0.152408,0.975409,Sierra Leone
221,Chad,2018.0,30.529295,0.022398,0.111988,0.716726,Chad
222,Central African Republic,2018.0,30.021749,0.015255,0.076277,0.488174,Central African Republic
223,Ethiopia,2018.0,29.815726,0.012356,0.061781,0.395399,Ethiopia


In [ ]:
water

In [ ]:
import country_converter as coco

# Step 1: Strip whitespace and ensure it's string
water['country'] = water['country'].astype(str).str.strip()

# Step 2: Safe conversion (no crash)
converted_names = []
for name in water['country']:
    try:
        converted = coco.convert(names=name, to='name_short', not_found=None)
        converted_names.append(converted)
    except Exception as e:
        print(f"Error converting {name}: {e}")
        converted_names.append(None)

# Step 3: Create cleaned column
water['country_c'] = converted_names

# Step 4: Show what didn't match
unmatched = water[water['country_c'].isna()]
print("❗ Unmatched country names:", unmatched['country'].unique())

# Step 5: Manual fixes (add to this as needed)
manual_fixes = {
    'Congo': 'Congo, Rep.',
    'Channel Islands': 'United Kingdom',
    'Samoa': 'Samoa'
}
water['country_c'] = water['country_c'].fillna(water['country'].map(manual_fixes))


In [ ]:
uw = pd.merge(under, water, on = 'country_c', how = 'inner')

In [ ]:
import country_converter as coco

def clean_country_names(pol, column='country', new_column='country_cleaned', manual_fixes=None):
    # Step 1: Basic cleanup
    pol['country'] = pol['country'].astype(str).str.strip()
    
    # Step 2: Try to convert names safely
    converted_names = []
    for name in pol['country']:
        try:
            converted = coco.convert(names=name, to='name_short', not_found=None)
            converted_names.append(converted)
        except Exception as e:
            print(f"Error converting {name}: {e}")
            converted_names.append(None)
    
    # Step 3: Assign new column
    pol['country_cleaned'] = converted_names
    
    # Step 4: Handle unmatched with manual fixes
    if manual_fixes is not None:
        pol['country_cleaned'] = pol['country_cleaned'].fillna(pol['country'].map(manual_fixes))
    
    # Step 5: Optional feedback
    unmatched = pol[pol['country_cleaned'].isna()]
    if not unmatched.empty:
        print("❗ Unmatched country names:", unmatched[column].unique())
    
    return df


In [844]:
water

,country,year,water_pct,norm_water,norm_water_withpts,norm_water_withpts_log5,country_c
0,Gibraltar,2018.0,100.000000,1.000000,5.000000,32.000000,Gibraltar
1,Monaco,2018.0,100.000000,1.000000,5.000000,32.000000,Monaco
2,Singapore,2018.0,100.000000,1.000000,5.000000,32.000000,Singapore
3,Liechtenstein,2018.0,100.000000,1.000000,5.000000,32.000000,Liechtenstein
4,San Marino,2018.0,100.000000,1.000000,5.000000,32.000000,San Marino
...,...,...,...,...,...,...,...
220,Sierra Leone,2018.0,31.103751,0.030482,0.152408,0.975409,Sierra Leone
221,Chad,2018.0,30.529295,0.022398,0.111988,0.716726,Chad
222,Central African Republic,2018.0,30.021749,0.015255,0.076277,0.488174,Central African Republic
223,Ethiopia,2018.0,29.815726,0.012356,0.061781,0.395399,Ethiopia


In [846]:
import country_converter as coco

# Manual fixes (expand this as needed)
manual_fixes = {
    'Congo': 'Congo, Rep.',
    'Channel Islands': 'United Kingdom',
    'Samoa': 'Samoa',
    'Ivory Coast': "Côte d'Ivoire",
    'Syria': 'Syrian Arab Republic',
    'Palestine': 'Palestine, State of',
    # Add more as needed!
}

def clean_country_names(df, column='country', new_column='country_cleaned', manual_fixes=None):
    df[column] = df[column].astype(str).str.strip()
    
    converted_names = []
    for name in df[column]:
        try:
            converted = coco.convert(names=name, to='name_short', not_found=None)
            converted_names.append(converted)
        except Exception as e:
            print(f"Error converting {name}: {e}")
            converted_names.append(None)
    
    df[new_column] = converted_names

    if manual_fixes is not None:
        df[new_column] = df[new_column].fillna(df[column].map(manual_fixes))
    
    unmatched = df[df[new_column].isna()]
    if not unmatched.empty:
        print("❗ Unmatched country names in `pol`:", unmatched[column].unique())
    
    return df

# ✅ Clean the `pol` dataframe
pol = clean_country_names(pol, manual_fixes=manual_fixes)


In [847]:
pol

,country,year,pm_value,norm_pm_value,norm_with_pts,norm_with_pts_log5,country_c,country_cleaned
0,Bahamas,2014.5,5.305895,0.000000,5.000000,0.000000,Bahamas,Bahamas
1,Finland,2014.5,6.002784,0.011293,4.943533,0.361392,Finland,Finland
2,Iceland,2014.5,6.296681,0.016056,4.919719,0.513800,Iceland,Iceland
3,Sweden,2014.5,6.532031,0.019870,4.900649,0.635848,Sweden,Sweden
4,Niue,2014.5,6.553944,0.020225,4.898873,0.647212,Niue,Niue
...,...,...,...,...,...,...,...,...
190,Egypt,2014.5,61.554086,0.911535,0.442326,29.169112,Egypt,Egypt
191,Qatar,2014.5,62.362459,0.924635,0.376826,29.588317,Qatar,Qatar
192,Tajikistan,2014.5,63.495392,0.942995,0.285026,30.175832,Tajikistan,Tajikistan
193,Kuwait,2014.5,65.130785,0.969497,0.152514,31.023912,Kuwait,Kuwait


In [848]:
pol_water = pd.merge(pol, water, on = 'country_c', how = 'inner')

In [849]:
pol_water

,country_x,year_x,pm_value,norm_pm_value,norm_with_pts,norm_with_pts_log5,country_c,country_cleaned,country_y,year_y,water_pct,norm_water,norm_water_withpts,norm_water_withpts_log5
0,Bahamas,2014.5,5.305895,0.000000,5.000000,0.000000,Bahamas,Bahamas,Bahamas,2016.5000,98.879833,0.984237,4.921184,31.495579
1,Finland,2014.5,6.002784,0.011293,4.943533,0.361392,Finland,Finland,Finland,2018.0000,99.803081,0.997229,4.986145,31.911326
2,Iceland,2014.5,6.296681,0.016056,4.919719,0.513800,Iceland,Iceland,Iceland,2018.0000,99.644100,0.994992,4.974959,31.839735
3,Sweden,2014.5,6.532031,0.019870,4.900649,0.635848,Sweden,Sweden,Sweden,2018.0000,99.694980,0.995708,4.978539,31.862647
4,Niue,2014.5,6.553944,0.020225,4.898873,0.647212,Niue,Niue,Niue,2018.0000,95.770680,0.940484,4.702422,30.095500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,Saudi Arabia,2014.5,57.001537,0.837758,0.811210,26.808257,Saudi Arabia,Saudi Arabia,Saudi Arabia,2018.5000,99.328363,0.990549,4.952743,31.697556
187,Egypt,2014.5,61.554086,0.911535,0.442326,29.169112,Egypt,Egypt,Egypt,2017.5625,94.703870,0.925472,4.627360,29.615106
188,Qatar,2014.5,62.362459,0.924635,0.376826,29.588317,Qatar,Qatar,Qatar,2018.0000,98.433561,0.977957,4.889784,31.294619
189,Tajikistan,2014.5,63.495392,0.942995,0.285026,30.175832,Tajikistan,Tajikistan,Tajikistan,2018.0000,69.629541,0.572622,2.863111,18.323913


In [850]:
# ccefmlps

import country_converter as coco

under['country_c'] = coco.convert(names=under['country'], to='name_short', not_found=None)


In [851]:
regions_to_drop = ['Melanesia', 'Polynesia']
under = under[~under['country'].isin(regions_to_drop)]


In [852]:
under['under_pts_log32'] = under['norm_undernourish'] * 32

In [876]:
uw = pd.merge(under, water, on = 'country_c', how = 'inner')

In [878]:
ccefmlpsuw = pd.merge(ccefmlps, uw, on = 'country_c', how = 'inner')
ccefmlpsuw['ccefmlpsuw_pts'] = ccefmlpsuw['under_pts_log32'] + ccefmlpsuw['norm_water_withpts_log5'] + ccefmlpsuw['ccefmlps_pts']

In [880]:
san = pd.read_csv(r"C:\Users\eabro\Documents\NSS\Python\Projects\Utopia\EmilyBroh_Capstone\emilybroh_capstone\Category_A\sanitize_withallpts_onlycountries.csv")

In [882]:
import country_converter as coco

# Manual fixes for countries not properly handled by coco
manual_fixes = {
    'Channel Islands': 'United Kingdom',
    'Samoa': 'Samoa',
    'Congo': 'Congo, Rep.',
}

# List of region-level entries to remove
regions_to_drop = [
    'Northern America',
    'Oceania',
    'Europe',
    'Americas',
    'Asia',
    'Africa'
]

# Clean function to standardize country names
def clean_country_names(df, column='country', new_column='country_c', manual_fixes=None, drop_regions=None):
    df[column] = df[column].astype(str).str.strip()
    
    converted_names = []
    for name in df[column]:
        try:
            converted = coco.convert(names=name, to='name_short', not_found=None)
            converted_names.append(converted)
        except Exception:
            converted_names.append(None)
    
    df[new_column] = converted_names

    if manual_fixes:
        df[new_column] = df[new_column].fillna(df[column].map(manual_fixes))
    
    if drop_regions:
        df = df[~df[column].isin(drop_regions)]

    unmatched = df[df[new_column].isna()]
    if not unmatched.empty:
        print("❗ Unmatched country names:", unmatched[column].unique())

    return df

# ✅ Apply to the `san` dataframe
san = clean_country_names(
    san,
    column='country',
    new_column='country_c',
    manual_fixes=manual_fixes,
    drop_regions=regions_to_drop
)



Channel Islands not found in regex


In [883]:
ccefmlpsuw

,country_c,ccefmlps_pts,country_x,year_x,pct_undernourish,norm_undernourish,norm_with_pts,under_pts_log32,country_y,year_y,water_pct,norm_water,norm_water_withpts,norm_water_withpts_log5,ccefmlpsuw_pts
0,Kuwait,244.307711,Kuwait,2017.5,2.50,1.000000,5.000000,32.000000,Kuwait,2018.000000,100.000000,1.000000,5.000000,32.000000,308.307711
1,United Arab Emirates,236.740303,United Arab Emirates,2017.5,3.84,0.975098,4.875488,31.203122,United Arab Emirates,2018.000000,99.973241,0.999623,4.998117,31.987950,299.931376
2,South Korea,230.185262,South Korea,2017.5,2.50,1.000000,5.000000,32.000000,South Korea,2018.000000,99.289160,0.989997,4.949985,31.679903,293.865165
3,Italy,227.573026,Italy,2017.5,2.50,1.000000,5.000000,32.000000,Italy,2018.000000,96.713262,0.953749,4.768743,30.519953,290.092979
4,Israel,225.434466,Israel,2017.5,2.50,1.000000,5.000000,32.000000,Israel,2018.000000,99.764349,0.996684,4.983419,31.893884,289.328350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,DR Congo,71.424128,Democratic Republic of Congo,2017.5,33.13,0.430775,2.153875,13.784798,DR Congo,2018.000000,28.937663,0.000000,0.000000,0.000000,85.208926
128,Sierra Leone,67.860229,Sierra Leone,2017.5,26.33,0.557146,2.785728,17.828656,Sierra Leone,2018.000000,31.103751,0.030482,0.152408,0.975409,86.664294
129,Somalia,63.062986,Somalia,2017.5,56.31,0.000000,0.000000,0.000000,Somalia,2018.000000,51.950023,0.323833,1.619167,10.362670,73.425656
130,Chad,40.441653,Chad,2017.5,28.55,0.515889,2.579446,16.508456,Chad,2018.000000,30.529295,0.022398,0.111988,0.716726,57.666834


In [884]:
ccefmlpssuw = pd.merge(ccefmlpsuw, san, on = 'country_c', how = 'inner')
ccefmlpssuw

,country_c,ccefmlps_pts,country_x,year_x,pct_undernourish,norm_undernourish,norm_with_pts,under_pts_log32,country_y,year_y,...,norm_water,norm_water_withpts,norm_water_withpts_log5,ccefmlpsuw_pts,country,year,sanitation_value,norm_sanitize,norm_sanitize_withpts5,norm_sanitize_withpts_log5
0,Kuwait,244.307711,Kuwait,2017.5,2.50,1.000000,5.000000,32.000000,Kuwait,2018.000000,...,1.000000,5.000000,32.000000,308.307711,Kuwait,2018.0,100.000000,1.000000,5.000000,32.000000
1,United Arab Emirates,236.740303,United Arab Emirates,2017.5,3.84,0.975098,4.875488,31.203122,United Arab Emirates,2018.000000,...,0.999623,4.998117,31.987950,299.931376,United Arab Emirates,2018.0,99.793913,0.997551,4.987753,31.921618
2,South Korea,230.185262,South Korea,2017.5,2.50,1.000000,5.000000,32.000000,South Korea,2018.000000,...,0.989997,4.949985,31.679903,293.865165,Republic of Korea,2018.0,99.866420,0.998412,4.992062,31.949195
3,Italy,227.573026,Italy,2017.5,2.50,1.000000,5.000000,32.000000,Italy,2018.000000,...,0.953749,4.768743,30.519953,290.092979,Italy,2018.0,99.985911,0.999833,4.999163,31.994641
4,Israel,225.434466,Israel,2017.5,2.50,1.000000,5.000000,32.000000,Israel,2018.000000,...,0.996684,4.983419,31.893884,289.328350,Israel,2018.0,99.948734,0.999391,4.996953,31.980502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,DR Congo,71.424128,Democratic Republic of Congo,2017.5,33.13,0.430775,2.153875,13.784798,DR Congo,2018.000000,...,0.000000,0.000000,0.000000,85.208926,Democratic Republic of the Congo,2018.0,35.353409,0.231647,1.158234,7.412698
128,Sierra Leone,67.860229,Sierra Leone,2017.5,26.33,0.557146,2.785728,17.828656,Sierra Leone,2018.000000,...,0.030482,0.152408,0.975409,86.664294,Sierra Leone,2018.0,52.621150,0.436882,2.184408,13.980208
129,Somalia,63.062986,Somalia,2017.5,56.31,0.000000,0.000000,0.000000,Somalia,2018.000000,...,0.323833,1.619167,10.362670,73.425656,Somalia,2018.0,54.008085,0.453366,2.266829,14.507707
130,Chad,40.441653,Chad,2017.5,28.55,0.515889,2.579446,16.508456,Chad,2018.000000,...,0.022398,0.111988,0.716726,57.666834,Chad,2018.0,17.432141,0.018645,0.093223,0.596629


In [885]:
ccefmlpssuw['ccefmlpssuw_pts'] = ccefmlpssuw['norm_sanitize_withpts_log5'] + ccefmlpssuw['ccefmlps_pts']

In [930]:
ccefmlpssuw_points = ccefmlpssuw[['country_c', 'ccefmlpssuw_pts']]
cat_aa = ccefmlpssuw_points

In [920]:
road = pd.read_csv(r"C:\Users\eabro\Documents\NSS\Python\Projects\Utopia\EmilyBroh_Capstone\emilybroh_capstone\Category_A\road_withpts_onlycountries.csv")

In [924]:
road['country_c'] = coco.convert(names= road['country'], to='name_short', not_found=None)

In [926]:
cat_a = pd.merge(cat_a, road, on = 'country_c', how = 'inner' )

In [934]:
cat_a['ccefmlpssuwt'] = cat_a

,country_c,ccefmlpssuw_pts,country,year,death_traffic,norm_traffic,traffic_pts
0,Kuwait,276.307711,Kuwait,2014.666667,18.100000,0.585326,18.730420
1,United Arab Emirates,268.661922,United Arab Emirates,2014.666667,15.466667,0.650453,20.814509
2,South Korea,262.134457,South Korea,2014.666667,10.966667,0.761748,24.375927
3,Italy,259.567668,Italy,2014.666667,6.166667,0.880462,28.174773
4,Israel,257.414968,Israel,2014.666667,4.333333,0.925804,29.625721
...,...,...,...,...,...,...,...
127,DR Congo,78.836825,Democratic Republic of Congo,2014.666667,34.800000,0.172300,5.513603
128,Sierra Leone,81.840437,Sierra Leone,2014.666667,31.133333,0.262984,8.415499
129,Somalia,77.570693,Somalia,2014.666667,27.433333,0.354493,11.343776
130,Chad,41.038281,Chad,2014.666667,27.600000,0.350371,11.211871


In [892]:
ccefmlpssuw_points.to_csv('Points_Category_A_Complete0413.csv', index=False, header=True)

In [694]:
cat_b = pd.read_csv(r"C:\Users\eabro\Documents\NSS\Python\Projects\Utopia\EmilyBroh_Capstone\emilybroh_capstone\All_Categories_Points\Points_Category_B_Complete0412.csv")

In [706]:
cat_a_b = pd.merge(cat_a, cat_b, on = 'country_c', how = 'inner')
cat_a_b

,country_c,ccefmlpssuw_pts,country,all7pts
0,Kuwait,276.307711,Kuwait,80.749510
1,United Arab Emirates,268.661922,United Arab Emirates,78.757195
2,South Korea,262.134457,South Korea,84.819722
3,Italy,259.567668,Italy,81.702278
4,Israel,257.414968,Israel,78.594187
...,...,...,...,...
88,Tanzania,107.754518,Tanzania,66.587901
89,Uganda,103.499358,Uganda,64.979462
90,Haiti,99.887952,Haiti,52.333273
91,Malawi,94.629250,Malawi,70.194156


In [710]:
cat_a_b['a_b_pts'] = cat_a_b['ccefmlpssuw_pts'] +cat_a_b['all7pts']

In [702]:
# ccefmlps

import country_converter as coco

cat_b['country_c'] = coco.convert(names=cat_b['country'], to='name_short', not_found=None)


In [714]:
cat_a_b = cat_a_b.sort_values('a_b_pts', ascending = False)

In [962]:
catab

,norm_crime_rate_log16,country_c,norm_gini_log16,homicide_norm_withpts_log16,norm_vac_withpts_log16,ihvg_pts,healthgdp_norm_withpts_log5,gini_log16,terror_log16,ineq_terror,ihhuvgtc_pts,ccefmlpssuw_pts
0,13.095168,Switzerland,13.132946,15.850197,13.783205,53.667652,9.012299,13.132946,15.990344,29.123290,104.898410,256.147570
1,11.385066,Norway,13.055823,15.818452,15.281173,56.504032,7.954588,13.055823,15.993315,29.049139,104.892825,253.184199
2,12.860908,Netherlands,13.756746,15.812411,14.411263,53.170516,8.645222,13.756746,15.950979,29.707726,104.384372,255.104394
3,12.743777,Czechia,16.000000,15.792009,15.305563,53.318915,5.899716,16.000000,15.988116,31.988116,103.950525,255.410652
4,12.158126,Austria,12.339092,15.812785,14.072937,51.102780,8.741722,12.339092,15.991830,28.330922,100.333550,255.737532
...,...,...,...,...,...,...,...,...,...,...,...,...
98,3.513909,Nigeria,8.377970,9.044233,NaN,NaN,1.329234,8.377970,11.842912,20.220882,NaN,101.796284
99,2.951684,Yemen,2.157319,14.671193,4.022372,NaN,2.736424,2.157319,16.000000,18.157319,NaN,170.174354
100,2.951684,Yemen,2.157319,14.671193,4.022372,NaN,2.736424,2.157319,16.000000,18.157319,NaN,170.174354
101,2.951684,Yemen,2.157319,14.671193,4.022372,NaN,2.736424,2.157319,11.541361,13.698680,NaN,170.174354


In [722]:
(7*16) + (11*32)

464

In [894]:
# 4/13
cat_a = pd.read_csv(r"C:\Users\eabro\Documents\NSS\Python\Projects\Utopia\EmilyBroh_Capstone\emilybroh_capstone\All_Categories_Points\Points_Category_A_Complete0413.csv")

In [898]:
ccefmlpssuw

,country_c,ccefmlps_pts,country_x,year_x,pct_undernourish,norm_undernourish,norm_with_pts,under_pts_log32,country_y,year_y,...,norm_water_withpts,norm_water_withpts_log5,ccefmlpsuw_pts,country,year,sanitation_value,norm_sanitize,norm_sanitize_withpts5,norm_sanitize_withpts_log5,ccefmlpssuw_pts
0,Kuwait,244.307711,Kuwait,2017.5,2.50,1.000000,5.000000,32.000000,Kuwait,2018.000000,...,5.000000,32.000000,308.307711,Kuwait,2018.0,100.000000,1.000000,5.000000,32.000000,276.307711
1,United Arab Emirates,236.740303,United Arab Emirates,2017.5,3.84,0.975098,4.875488,31.203122,United Arab Emirates,2018.000000,...,4.998117,31.987950,299.931376,United Arab Emirates,2018.0,99.793913,0.997551,4.987753,31.921618,268.661922
2,South Korea,230.185262,South Korea,2017.5,2.50,1.000000,5.000000,32.000000,South Korea,2018.000000,...,4.949985,31.679903,293.865165,Republic of Korea,2018.0,99.866420,0.998412,4.992062,31.949195,262.134457
3,Italy,227.573026,Italy,2017.5,2.50,1.000000,5.000000,32.000000,Italy,2018.000000,...,4.768743,30.519953,290.092979,Italy,2018.0,99.985911,0.999833,4.999163,31.994641,259.567668
4,Israel,225.434466,Israel,2017.5,2.50,1.000000,5.000000,32.000000,Israel,2018.000000,...,4.983419,31.893884,289.328350,Israel,2018.0,99.948734,0.999391,4.996953,31.980502,257.414968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,DR Congo,71.424128,Democratic Republic of Congo,2017.5,33.13,0.430775,2.153875,13.784798,DR Congo,2018.000000,...,0.000000,0.000000,85.208926,Democratic Republic of the Congo,2018.0,35.353409,0.231647,1.158234,7.412698,78.836825
128,Sierra Leone,67.860229,Sierra Leone,2017.5,26.33,0.557146,2.785728,17.828656,Sierra Leone,2018.000000,...,0.152408,0.975409,86.664294,Sierra Leone,2018.0,52.621150,0.436882,2.184408,13.980208,81.840437
129,Somalia,63.062986,Somalia,2017.5,56.31,0.000000,0.000000,0.000000,Somalia,2018.000000,...,1.619167,10.362670,73.425656,Somalia,2018.0,54.008085,0.453366,2.266829,14.507707,77.570693
130,Chad,40.441653,Chad,2017.5,28.55,0.515889,2.579446,16.508456,Chad,2018.000000,...,0.111988,0.716726,57.666834,Chad,2018.0,17.432141,0.018645,0.093223,0.596629,41.038281


In [900]:
catb = pd.read_csv(r"C:\Users\eabro\Documents\NSS\Python\Projects\Utopia\EmilyBroh_Capstone\emilybroh_capstone\All_Categories_Points\Points_Category_B_Method2.csv")

In [916]:
cat_a

,country_c,ccefmlpssuw_pts
0,Kuwait,276.307711
1,United Arab Emirates,268.661922
2,South Korea,262.134457
3,Italy,259.567668
4,Israel,257.414968
...,...,...
127,DR Congo,78.836825
128,Sierra Leone,81.840437
129,Somalia,77.570693
130,Chad,41.038281


In [948]:

cat_a.columns

Index(['country_c', 'ccefmlpssuw_pts', 'country', 'year', 'death_traffic',
       'norm_traffic'],
      dtype='object')

In [954]:
cat_a = cat_a.sort_values('ccefmlpssuw_pts', ascending = False)

In [958]:
cat_a.head(25)

,country_c,ccefmlpssuw_pts,country,year,death_traffic,norm_traffic
0,Kuwait,276.307711,Kuwait,2014.666667,18.100000,0.585326
1,United Arab Emirates,268.661922,United Arab Emirates,2014.666667,15.466667,0.650453
2,South Korea,262.134457,South Korea,2014.666667,10.966667,0.761748
3,Italy,259.567668,Italy,2014.666667,6.166667,0.880462
4,Israel,257.414968,Israel,2014.666667,4.333333,0.925804
5,Japan,257.099960,Japan,2014.666667,4.366667,0.924979
6,Egypt,256.344186,Egypt,2014.666667,11.833333,0.740313
7,Switzerland,256.147570,Switzerland,2014.666667,3.200000,0.953833
8,Malta,255.800562,Malta,2014.666667,3.600000,0.943941
9,Austria,255.737532,Austria,2014.666667,5.800000,0.889530


In [1001]:
# April 16th: Cleaned up final 5 datasets for category A. Joining them together now and determining next steps for missing countries.

In [1036]:
polu = pd.read_csv(r"C:\Users\eabro\Documents\NSS\Python\Projects\Utopia\EmilyBroh_Capstone\emilybroh_capstone\Category_A\Pollution_20152019.csv")
basics= pd.read_csv(r"C:\Users\eabro\Documents\NSS\Python\Projects\Utopia\EmilyBroh_Capstone\emilybroh_capstone\Category_A\Basic_san20172022.csv")
water = pd.read_csv(r"C:\Users\eabro\Documents\NSS\Python\Projects\Utopia\EmilyBroh_Capstone\emilybroh_capstone\Category_A\Basicwater_20162022.csv")
calorie = pd.read_csv(r"C:\Users\eabro\Documents\NSS\Python\Projects\Utopia\EmilyBroh_Capstone\emilybroh_capstone\Category_A\Calorie_20192023.csv")
life = pd.read_csv(r"C:\Users\eabro\Documents\NSS\Python\Projects\Utopia\EmilyBroh_Capstone\emilybroh_capstone\Category_A\LifeExpect_2018.csv")

In [1038]:
polu.shape, basics.shape, water.shape, calorie.shape, life.shape

((195, 5), (228, 11), (231, 6), (145, 5), (237, 10))

In [1048]:
polu['country_c'] = coco.convert(names= polu['country'], to='name_short', not_found=None)
calorie['country_c'] = coco.convert(names= calorie['country'], to='name_short', not_found=None)

In [1102]:
ps = pd.merge(polu, basics, on = 'country_c', how = 'inner')
ps_o = pd.merge(polu, basics, on = 'country_c', how = 'outer')

In [1104]:
psw = pd.merge(ps, water, on = 'country_c', how = 'inner')
psw_o = pd.merge(ps, water, on = 'country_c', how = 'outer')

In [1112]:
psw  = psw[['country_c', 'pm25', 'norm_pol2', 'pol_log32_n', 'safe',  'atleast_basicsan', 'norm_basicsan',
       'basicsan_log32_n', 'basicw', 'norm_water',
       'water_log32_n']]

In [1110]:
pswc = pd.merge(psw, calorie, on = 'country_c', how = 'inner')


MergeError: Passing 'suffixes' which cause duplicate columns {'year_x', 'country_x'} is not allowed.

In [1120]:
pswcl = pd.merge(pswc, life, on = 'country_c', how = 'inner')
pswcl_o = pd.merge(pswc, life, on = 'country_c', how = 'outer')

In [1068]:
pswcl = pswcl[['country_c', 'pm25', 'norm_pol2', 'pol_log32_n', 'safe',  'atleast_basicsan', 'norm_basicsan',
       'basicsan_log32_n', 'basicw', 'norm_water',
       'water_log32_n', 'calorie',
       'norm_calorie', 'calorie_log32_n', 'avg_life', 'norm_life_expect', 'norm_pts_log5', 'lifeexpect_log32_n']]

In [ ]:
pswcl = pswcl.drop('norm_pts_log5', axis = 1)
pswcl_o = pswcl.drop('norm_pts_log5', axis = 1)

In [1082]:
pswcl['category_a_pts'] = pswcl['pol_log32_n'] + pswcl['basicsan_log32_n'] + pswcl['water_log32_n'] + pswcl['calorie_log32_n'] + pswcl['lifeexpect_log32_n']


In [1126]:
pswcl_o['category_a_pts'] = pswcl_o['pol_log32_n'] + pswcl_o['basicsan_log32_n'] + pswcl_o['water_log32_n'] + pswcl_o['calorie_log32_n'] + pswcl_o['lifeexpect_log32_n']

In [1090]:
pswcl = pswcl.sort_values('category_a_pts', ascending = False).reset_index()

In [1100]:
pswcl.to_csv('Points_Category_A_Inner_Complete0416.csv', index=False, header=True)

In [1128]:
pswcl_o = pswcl_o.sort_values('category_a_pts', ascending = False).reset_index()

In [1433]:
pswcl_o.to_csv('Points_Category_A_Outer_Complete0416.csv', index=False, header=True)


,country_c,pm25,norm_pol2,pol_log32_n,safe,atleast_basicsan,norm_basicsan,basicsan_log32_n,basicw,norm_water,...,norm_calorie,calorie_log32_n,country_y,year_y,avg_life,median_life,avg_med_life,norm_life_expect,norm_with_pts,lifeexpect_log32_n
0,Finland,5.525910,0.999542,31.985359,89.609373,99.999956,1.000000,31.999985,100.000000,1.000000,...,0.994924,31.837563,Finland,2018,81.49,81.50,81.49,0.898182,4.490909,28.741818
1,Iceland,5.909562,0.993123,31.779924,0.000000,1.219210,0.012192,0.390147,100.000000,1.000000,...,0.994924,31.837563,Iceland,2018,82.32,82.43,82.38,0.918303,4.591515,29.385697
2,Sweden,6.046158,0.990837,31.706780,95.502112,99.589598,0.995896,31.868670,99.715433,0.995577,...,0.965736,30.903553,Sweden,2018,82.59,82.44,82.51,0.924848,4.624242,29.595152
3,Estonia,6.378954,0.985268,31.528577,90.421816,99.770448,0.997704,31.926543,99.972120,0.999567,...,0.958122,30.659898,Estonia,2018,78.06,78.17,78.12,0.815030,4.075152,26.080970
4,Norway,6.428530,0.984438,31.502030,77.876965,100.000002,1.000000,32.000000,99.999999,1.000000,...,0.984772,31.512690,Norway,2018,82.64,82.63,82.64,0.926061,4.630303,29.633939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,Pakistan,53.631344,0.194568,6.226183,0.000000,10.128144,0.101281,3.241006,90.187921,0.847496,...,0.875635,28.020305,Pakistan,2018,66.19,65.94,66.07,0.527273,2.636364,16.872727
139,Nigeria,56.813564,0.141318,4.522189,30.311561,61.586757,0.615868,19.707762,74.725992,0.607179,...,0.229695,7.350254,Nigeria,2018,52.82,52.67,52.75,0.203152,1.015758,6.500848
140,Cameroon,58.351032,0.115591,3.698916,0.000000,17.119519,0.171195,5.478246,67.998398,0.502615,...,0.601523,19.248731,Cameroon,2018,60.95,61.15,61.05,0.400242,2.001212,12.807758
141,Tajikistan,59.523014,0.095980,3.071351,0.000000,2.623740,0.026237,0.839597,80.649989,0.699252,...,0.906091,28.994924,Tajikistan,2018,70.25,69.98,70.11,0.625697,3.128485,20.022303


In [1132]:
pswcl_o

,index,country_c,pm25,norm_pol2,pol_log32_n,safe,atleast_basicsan,norm_basicsan,basicsan_log32_n,basicw,norm_water,water_log32_n,calorie,norm_calorie,calorie_log32_n,avg_life,norm_life_expect,norm_pts_log5,lifeexpect_log32_n,category_a_pts
0,157,Norway,6.428530,0.984438,31.502030,77.876965,100.000002,1.000000,32.000000,99.999999,1.000000,31.999999,1.2,0.984772,31.512690,82.64,0.926061,29.633939,29.633939,156.648659
1,70,Finland,5.525910,0.999542,31.985359,89.609373,99.999956,1.000000,31.999985,100.000000,1.000000,32.000000,0.4,0.994924,31.837563,81.49,0.898182,28.741818,28.741818,156.564726
2,11,Australia,7.241084,0.970842,31.066929,95.562399,99.999826,0.999998,31.999944,99.969854,0.999531,31.985007,2.5,0.968274,30.984772,83.06,0.936242,29.959758,29.959758,155.996409
3,203,Sweden,6.046158,0.990837,31.706780,95.502112,99.589598,0.995896,31.868670,99.715433,0.995577,31.858468,2.7,0.965736,30.903553,82.59,0.924848,29.595152,29.595152,155.932623
4,204,Switzerland,9.935088,0.925761,29.624362,99.699506,100.000000,1.000000,31.999999,100.000000,1.000000,32.000000,1.1,0.986041,31.553299,83.34,0.943030,30.176970,30.176970,155.354630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,230,Venezuela,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.47,0.679515,21.744485,21.744485,NaN
233,232,Wallis and Futuna Islands,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.78,0.808242,25.863758,25.863758,NaN
234,233,Western Sahara,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.08,0.621576,19.890424,19.890424,NaN
235,234,Yemen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.19,0.551515,17.648485,17.648485,NaN


In [1138]:
pswcl_o.isna('pol_log32_n')

TypeError: DataFrame.isna() takes 1 positional argument but 2 were given

In [1288]:
missing_pol = pswcl_o.loc[pd.isna(pswcl_o['pol_log32_n'])]
missing_san = pswcl_o.loc[pd.isna(pswcl_o['basicsan_log32_n'])]
missing_water = pswcl_o.loc[pd.isna(pswcl_o['water_log32_n'])]
missing_calorie = pswcl_o.loc[pd.isna(pswcl_o['calorie_log32_n'])]
missing_life = pswcl_o.loc[pd.isna(pswcl_o['lifeexpect_log32_n'])]

In [1164]:
missing_pol_vc = missing_pol.value_counts('country_c').reset_index()

In [1290]:
missing_san_vc = missing_san.value_counts('country_c').reset_index()


In [1278]:
missing_water_vc = missing_water.value_counts('country_c').reset_index()
missing_calorie_vc = missing_calorie.value_counts('country_c').reset_index()
missing_life_vc = missing_life.value_counts('country_c').reset_index()

In [1304]:
missing_water_vc.columns = ['country_c', 'missing_water']

In [1306]:
missing_calorie_vc.columns =  ['country_c', 'missing_calorie']

In [1308]:
missing_san_vc.columns =  ['country_c', 'missing_san']

In [1310]:
missing_life_vc.columns =  ['country_c', 'missing_life']

In [1312]:
missing_pol_vc.columns =  ['country_c', 'missing_pol']

In [1368]:
missing_ps = pd.merge(missing_pol_vc, missing_san_vc, on = ['country_c'], how = 'outer')
missing_ps

,country_c,missing_pol,missing_san
0,Afghanistan,1,1
1,American Samoa,1,1
2,Andorra,1,1
3,Anguilla,1,1
4,Antigua and Barbuda,1,1
...,...,...,...
89,Venezuela,1,1
90,Wallis and Futuna Islands,1,1
91,Western Sahara,1,1
92,Yemen,1,1


In [1370]:
missing_psw = pd.merge(missing_ps, missing_water_vc, on = ['country_c'], how = 'outer')


In [1372]:
#missing_psw = missing_psw[['country_c', 'count']]

In [1374]:
missing_pswc = pd.merge(missing_psw, missing_calorie_vc, on = 'country_c', how = 'outer')


In [1376]:
missing_pswcl = pd.merge(missing_pswc, missing_life_vc, on = 'country_c', how = 'outer')

In [1378]:
missing_pswcl = missing_pswcl.fillna(0)

In [1380]:
missing_pswcl['total_missing'] = missing_pswcl['missing_pol'] + missing_pswcl['missing_san'] + missing_pswcl['missing_water'] + missing_pswcl['missing_calorie'] +missing_pswcl['missing_life']

In [1382]:
missing_pswcl = missing_pswcl.sort_values('total_missing', ascending = False)

In [1406]:
missing_pswcl2 = missing_pswcl.loc[missing_pswcl['total_missing'] == 4.0]

In [1408]:
missing_pswcl2

,country_c,missing_pol,missing_san,missing_water,missing_calorie,missing_life,total_missing
0,Afghanistan,1,1,1,1,0.0,4.0
59,Palau,1,1,1,1,0.0,4.0
68,Singapore,1,1,1,1,0.0,4.0
67,Saudi Arabia,1,1,1,1,0.0,4.0
66,San Marino,1,1,1,1,0.0,4.0
...,...,...,...,...,...,...,...
29,Gibraltar,1,1,1,1,0.0,4.0
28,Georgia,1,1,1,1,0.0,4.0
27,French Polynesia,1,1,1,1,0.0,4.0
26,French Guiana,1,1,1,1,0.0,4.0


# Upon double and triple checking, all countries with any missing values at all are missing all 4 of the same values: Pollution, Sanitation, Water, and Calories. None of these countries are missing the life expectancy category.

In [1413]:
missing_pswcl.loc[missing_pswcl['country_c'] == 'Iran']

,country_c,missing_pol,missing_san,missing_water,missing_calorie,missing_life,total_missing
36,Iran,1,1,1,1,0.0,4.0


In [1417]:
pswcl3 = pd.merge(pswc, life, on = 'country_c', how = 'outer')

In [1427]:
pswcl3 = pswcl3[['country_c', 'pm25', 'norm_pol2', 'pol_log32_n', 'safe', 'atleast_basicsan', 'norm_basicsan', 'basicsan_log32_n', 'basicw','norm_water', 'water_log32_n','calorie','norm_calorie', 'calorie_log32_n', 'avg_life','norm_life_expect', 'lifeexpect_log32_n']]

In [1431]:
pswcl3.to_csv('Points_Category_A_Inner_withLife_0416.csv', index=False, header=True)

In [1437]:
pswcl['pswcl_pts'] = pswcl['pol_log32_n'] + pswcl['basicsan_log32_n'] + pswcl['water_log32_n'] + pswcl ['calorie_log32_n'] + pswcl['lifeexpect_log32_n']

In [1441]:
pswcl = pswcl.sort_values('pswcl_pts', ascending = False)

In [1455]:
pswcl2 = pswcl[['country_c', 'pm25','pol_log32_n',
       'atleast_basicsan', 'basicsan_log32_n', 'basicw',
       'norm_water', 'water_log32_n', 'calorie',
      'calorie_log32_n',  'avg_life',
       'lifeexpect_log32_n', 'pswcl_pts']]

In [1459]:
pswcl2.to_csv('Points_Category_A_Complete_withallpts_0416.csv', index=False, header=True)

In [1463]:
pswcl2

,country_c,pm25,pol_log32_n,atleast_basicsan,basicsan_log32_n,basicw,norm_water,water_log32_n,calorie,calorie_log32_n,avg_life,lifeexpect_log32_n,pswcl_pts
4,Norway,6.428530,31.502030,100.000002,32.000000,99.999999,1.000000,31.999999,1.2,31.512690,82.64,29.633939,156.648659
0,Finland,5.525910,31.985359,99.999956,31.999985,100.000000,1.000000,32.000000,0.4,31.837563,81.49,28.741818,156.564726
6,Australia,7.241084,31.066929,99.999826,31.999944,99.969854,0.999531,31.985007,2.5,30.984772,83.06,29.959758,155.996409
2,Sweden,6.046158,31.706780,99.589598,31.868670,99.715433,0.995577,31.858468,2.7,30.903553,82.59,29.595152,155.932623
16,Switzerland,9.935088,29.624362,100.000000,31.999999,100.000000,1.000000,32.000000,1.1,31.553299,83.34,30.176970,155.354630
...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,Cameroon,58.351032,3.698916,17.119519,5.478246,67.998398,0.502615,16.083684,31.4,19.248731,60.95,12.807758,57.317335
136,Niger,51.736746,7.240689,24.349297,7.791775,47.412132,0.182653,5.844900,22.9,22.700508,59.59,11.752727,55.330599
130,Chad,43.420190,11.693983,17.803308,5.697058,50.424345,0.229470,7.343054,19.6,24.040609,52.57,6.306909,55.081614
109,DR Congo,31.912296,17.856153,34.487549,11.036015,35.660282,0.000000,0.000000,62.1,6.781726,59.71,11.845818,47.519713
